# Reprojecting, stacking and colour-combination

On this page we'll reproject our pre-reduced exposures on a common pixel grid, stack them, and produce a colour image.

You can copy or write the code shown below in a script, or alternatively directly download this page as a {download}`jupyter notebook <./stack.ipynb>` file.


In [10]:

from pathlib import Path

import numpy as np
import astropy
import astropy.visualization
from astropy import units as u
%matplotlib ipympl
import matplotlib
from matplotlib import pyplot as plt
import ccdproc
#import photutils


In [4]:
# Location of your workdir (i.e., where the pre-reduced files have been written):
work_dir = Path("/Users/mtewes/Desktop/no_backup/workdir_2024-03-08")

# To make things easier to read, we only display those header keywords:
header_keywords = ["imagetyp", "filter", "exptime", "object"]


In [6]:

light_prered_dir = work_dir / "LIGHT_PRERED"
science_files = ccdproc.ImageFileCollection(light_prered_dir, keywords=header_keywords)

# Let's first get an overview of all available files:
science_files.summary

<Table masked=True length=18>
                       file                       imagetyp filter exptime object
                      str49                         str5    str1  float64  str4 
------------------------------------------------- -------- ------ ------- ------
2024-03-08_22-04-34_r_-10.00_60.00s_0000.new.fits    LIGHT      r    60.0   M 37
2024-03-08_22-05-37_r_-10.00_60.00s_0001.new.fits    LIGHT      r    60.0   M 37
2024-03-08_22-06-39_r_-10.00_60.00s_0002.new.fits    LIGHT      r    60.0   M 37
2024-03-08_22-07-43_g_-10.00_60.00s_0000.new.fits    LIGHT      g    60.0   M 37
2024-03-08_22-08-45_g_-10.00_60.00s_0001.new.fits    LIGHT      g    60.0   M 37
2024-03-08_22-09-47_g_-10.00_60.00s_0002.new.fits    LIGHT      g    60.0   M 37
2024-03-08_22-10-51_i_-10.00_60.00s_0000.new.fits    LIGHT      i    60.0   M 37
 2024-03-08_22-11-53_i_-9.90_60.00s_0001.new.fits    LIGHT      i    60.0   M 37
2024-03-08_22-12-55_i_-10.00_60.00s_0002.new.fits    LIGHT      i    60.0   M 37
2024-03-08_23-45-21_g_-10.00_60.00s_0003.new.fits    LIGHT      g    60.0   M 66
 2024-03-08_23-46-23_g_-9.90_60.00s_0004.new.fits    LIGHT      g    60.0   M 66
2024-03-08_23-47-25_g_-10.00_60.00s_0005.new.fits    LIGHT      g    60.0   M 66
2024-03-08_23-48-29_r_-10.00_60.00s_0003.new.fits    LIGHT      r    60.0   M 66
2024-03-08_23-49-31_r_-10.00_60.00s_0004.new.fits    LIGHT      r    60.0   M 66
2024-03-08_23-50-33_r_-10.00_60.00s_0005.new.fits    LIGHT      r    60.0   M 66
2024-03-08_23-51-38_i_-10.00_60.00s_0003.new.fits    LIGHT      i    60.0   M 66
2024-03-08_23-52-40_i_-10.00_60.00s_0004.new.fits    LIGHT      i    60.0   M 66
2024-03-08_23-53-42_i_-10.00_60.00s_0005.new.fits    LIGHT      i    60.0   M 66

In [7]:
# Where to write the reprojected files:
dest_dir = work_dir / "REPROJ"
dest_dir.mkdir(exist_ok=True)


Rerun the following three cells with all filters:

In [8]:
# Select object and filter:
selected_object = "M 37"
selected_filter = "i"

selected_science_files = science_files.filter(object=selected_object, filter=selected_filter)
selected_science_files.summary

<Table masked=True length=3>
                                                       file                                                       ...
                                                      str113                                                      ...
----------------------------------------------------------------------------------------------------------------- ...
/Users/mtewes/Desktop/no_backup/workdir_2024-03-08/LIGHT_PRERED/2024-03-08_22-10-51_i_-10.00_60.00s_0000.new.fits ...
 /Users/mtewes/Desktop/no_backup/workdir_2024-03-08/LIGHT_PRERED/2024-03-08_22-11-53_i_-9.90_60.00s_0001.new.fits ...
/Users/mtewes/Desktop/no_backup/workdir_2024-03-08/LIGHT_PRERED/2024-03-08_22-12-55_i_-10.00_60.00s_0002.new.fits ...

In [16]:
# Select first image as the target to project on (note: same image for *all* filters!)
target_image = ccdproc.CCDData.read(science_files.filter(object=selected_object).files[0])

# Looping over the images to do the reprojection (takes a while):
for ccd, filename in selected_science_files.ccds(ccd_kwargs={'unit': 'adu'}, return_fname=True):
    print(f"Reprojecting {filename}...")
    
    ccd = ccdproc.wcs_project(ccd, target_image.wcs)

    # Write to disk:
    ccd.data = ccd.data.astype('float32')
    ccd.write(dest_dir / filename, overwrite=True)


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Reprojecting 2024-03-08_22-10-51_i_-10.00_60.00s_0000.new.fits...


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Reprojecting 2024-03-08_22-11-53_i_-9.90_60.00s_0001.new.fits...


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Reprojecting 2024-03-08_22-12-55_i_-10.00_60.00s_0002.new.fits...


In [17]:
# Combine the reprojected images of that filter:

files_to_combine = ccdproc.ImageFileCollection(dest_dir).files_filtered(object=selected_object, filter=selected_filter, include_path=True)
ccd = ccdproc.combine(files_to_combine,
                        method='average', scale=None,
                        sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                        sigma_clip_func=np.ma.median, signma_clip_dev_func=astropy.stats.mad_std,
                        mem_limit=1e9
                    )

ccd.meta['combined'] = True
ccd.data = ccd.data.astype('float32') # Converts to float32 to save space
ccd.write(work_dir / f"combi_{selected_object}_{selected_filter}.fits", overwrite=True)


INFO:astropy:splitting each image into 3 chunks to limit memory usage to 1000000000.0 bytes.


INFO: splitting each image into 3 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]


## Colour-combination

This is the colour-combination algorithm used by SDSS.
It might not be "optimal" for pretty pictures (well that's a matter of taste), but it is scientifically interesting.

Todo: describe more, and also compare with simple RGB composite.

In [11]:

g_ccd = ccdproc.CCDData.read(work_dir / f"combi_{selected_object}_g.fits")
r_ccd = ccdproc.CCDData.read(work_dir / f"combi_{selected_object}_r.fits")
i_ccd = ccdproc.CCDData.read(work_dir / f"combi_{selected_object}_i.fits")

# i -> R
i_ccd.data *= 1.0
# r -> G
r_ccd.data *= 0.4
# g -> B
g_ccd.data *= 0.3

sky_levels = (np.nanmedian(i_ccd.data), np.nanmedian(r_ccd.data), np.nanmedian(g_ccd.data))

rgbimage = astropy.visualization.make_lupton_rgb(i_ccd.data, r_ccd.data, g_ccd.data,
                                                 minimum=1.0*np.array(sky_levels),
                                                 stretch=30, Q=5, 
                                                 filename=work_dir/f"combi_{selected_object}.jpg")

# Note that this both returns the image as a numpy array, and writes it to jpg.

In [12]:
fig = plt.figure(figsize=(10, 7))
ax = fig.subplots()
ax.imshow(rgbimage , origin="lower")
fig.tight_layout()
plt.show()

In [25]:

# If needed, one can also write the image manually for example with matplotlib:
#import matplotlib.image
#matplotlib.image.imsave("rgbimage.png", rgbimage, origin="lower")